In [23]:
from tqdm import tqdm
from openai import OpenAI
import os
from dotenv import load_dotenv
from huggingface_hub import login

In [24]:
load_dotenv()
login(os.getenv('HF_TOKEN'))

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv('OPENROUTER_API_KEY'),
)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


### Simple Classifier

In [ ]:
user_input = "Um — do you know where the world’s largest ice sheet is located today?"

classifier_input = f'''Please assess what personality best fits the following text. The categories are:  
- Formal  
- Casual  
- Confident  
- Hesitant  
- Analytical  
- Emotional  
- Optimistic  
- Pessimistic  

<text>  
{user_input}
</text>  

Please respond with a single word.'''

In [21]:
completion = client.chat.completions.create(
    model='google/gemini-2.5-flash',
    messages=[
        {
        "role": "user",
        "content": classifier_input,
        }
    ]
)

In [22]:
print(completion.choices[0].message.content)

Hesitant


### Classifier with logits

In [ ]:
user_input = "Um — do you know where the world’s largest ice sheet is located today?"

classifier_input = f'''Please assess what personality best fits the following text. The categories are:  
- Formal  
- Casual  
- Confident  
- Hesitant  
- Analytical  
- Emotional  
- Optimistic  
- Pessimistic  

<text>  
{user_input}
</text>  

Please respond with a single word.'''

In [30]:
completion = client.chat.completions.create(
    model='openai/gpt-4o-mini',
    messages=[
        {
        "role": "user",
        "content": classifier_input,
        }
    ],
    logprobs=True,
    top_logprobs=5,
)

In [31]:
print(completion.choices[0].message.content)

Hesitant


In [43]:
content = completion.choices[0].logprobs.content

In [48]:
content[0].top_logprobs

[TopLogprob(token='H', bytes=[72], logprob=-4.320199877838604e-07),
 TopLogprob(token='Cur', bytes=[67, 117, 114], logprob=-15.25),
 TopLogprob(token='Cas', bytes=[67, 97, 115], logprob=-15.75),
 TopLogprob(token=' hesitant', bytes=[32, 104, 101, 115, 105, 116, 97, 110, 116], logprob=-17.375),
 TopLogprob(token='P', bytes=[80], logprob=-17.75)]